In [24]:
import gym
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint


# todos

## Environment
- [done] get ships from contract
   * np.where + mod 
   * create a column `is_available` in `fleet` with fleet availability   
- [ ]create `contracts_df` from `create_contracts()`.
   * Columns : 
     * `start_port`
     * `ship_type`
     * `end_port`
     * `start_day` =  start_step + (steps needed for pick up == dm / 14 knots) ? probably 10 knots
     * `end_day` = start_day + max_distance in distance_matrix / 10 knots in steps
     * `value_in_$` = values_dict for each ship type
     * `assigned_ship` = [0,1,2] None, ship_1, ship_2 
- [ ] create `can_reach` function that calculates if a ship can make it in time for the contract
   * maybe implement `can_reach` with 10,12,14 knots? 



## Agent
- [ ] Action Space:
   * Choose contract Discrete 5 [0,1,2,3,4] choose zero, one, two, three or all contracts
   * For each contract: 
     * Choose ship Discrete 3 [0,1,2]
     * Choose speed Discrete 3 [10,12,14]
- [ ] 

In [121]:
# load data

ports_df = pd.read_csv('data/ports_10.csv')
distance_matrix = pd.read_csv('data/distance_matrix.csv')
fleet_df = pd.read_csv('data/fleet.csv')

In [9]:
fleet_df

,ship_name,ship_type,dwt,cii_threshold,cii_attained,current_port,current_speed,is_available
0,handy_1,handymax,40000,0,0,0,0,1
1,handy_2,handymax,40000,0,0,0,0,1
2,supra_1,supramax,50000,0,0,0,0,1
3,supra_2,supramax,50000,0,0,0,0,1
4,ultra_1,ultramax,60000,0,0,0,0,1
5,ultra_2,ultramax,60000,0,0,0,0,1
6,pana_1,panamax,70000,0,0,0,0,1
7,pana_2,panamax,70000,0,0,0,0,1
8,kamsar_1,kamsarmax,80000,0,0,0,0,1
9,kamsar_2,kamsarmax,80000,0,0,0,0,1


In [35]:
data_dict = {
    "fleet_path":'data/fleet.csv',
    "ports_path":'data/ports_10.csv',
    "dm_path" : 'data/distance_matrix.csv'
}

In [5]:
ports_df

,number,name,country,region,code,position,latitude,longitude,location
0,1,Shanghai,China,East Asia,CNSHA,"31° 15.500' N, 121° 32.600' E",31.258333,121.543333,Yangtze Delta
1,2,Singapore,Singapore,Southeast Asia,SGSIN,"1° 15.500' N, 103° 47.900' E",1.258330,103.798333,Singapore Strait
2,3,Xiamen,China,East Asia,CNXMN,"24° 27.100' N, 118° 4.300' E",24.451670,118.071667,Taiwan Strait
3,4,Dalian,China,East Asia,CNDLC,"38° 56.300' N, 121° 39.500' E",38.938330,121.658333,Yellow Sea
4,5,Guangzhou,China,East Asia,CNCAN,"23° 3.900' N, 113° 25.150' E",23.065000,113.419167,Pearl River Delta
5,6,Qingdao,China,East Asia,CNTAO,"36° 5.650' N, 120° 19.000' E",36.094170,120.316667,Yellow Sea
6,7,Busan,South Korea,East Asia,KRPUS,"35° 6.500' N, 129° 3.500' E",35.108330,129.058333,Korean Strait
7,8,Hong Kong,Hong Kong,East Asia,HKHKG,"22° 17.400' N, 114° 10.100' E",22.290000,114.168333,Pearl River Delta
8,9,Tianjin,China,East Asia,CNTXG,"38° 58.400' N, 117° 46.300' E",38.973330,117.771667,Yellow Sea
9,10,Kaohsiung,Taiwan,East Asia,TWKHH,"22° 34.800' N, 120° 18.000' E",22.580000,120.300000,Taiwan Strait


In [3]:
# set fleet at random ports

fleet_df['current_port'] =  np.random.randint(1,11,fleet.shape[0])
fleet_df

,vessel_name,vessel_type,dwt,cii_required,cii_attained,current_port,current_speed,is_available
0,handy_1,handymax,40000,0,0,6,0,1
1,handy_2,handymax,40000,0,0,7,0,1
2,supra_1,supramax,50000,0,0,5,0,1
3,supra_2,supramax,50000,0,0,5,0,1
4,ultra_1,ultramax,60000,0,0,10,0,1
5,ultra_2,ultramax,60000,0,0,7,0,1
6,pana_1,panamax,70000,0,0,4,0,1
7,pana_2,panamax,70000,0,0,6,0,1
8,kamsar_1,kamsarmax,80000,0,0,6,0,1
9,kamsar_2,kamsarmax,80000,0,0,1,0,1


In [ ]:
# df_contr_total = start_port, ship_type, end_port, start_day, end_day, value, assigned_ship
# 4*365 = 1460 rows
df_contr_total = pd.DataFrame(columns=['start_port','end_port','ship_type','start_day','end_day','assigned_ship','value','cargo_size','duration','distance'],
                            index = range(1,1461))
df_contr_total

# we will append df_contr_per_day to contracts 

In [195]:
df_contr_per_day = pd.DataFrame(columns=['start_port','end_port','ship_type','start_day','end_day','assigned_ship','value','cargo_size','duration','distance'],
                            index = range(0,4))

In [235]:
# create random contracts
def create_contrs(self=None, df=df_contr_per_day, dm = distance_matrix , step=1,seed=None):
        """
        A method that creates 4 random cargo contracts at each env step. \ 
        There are 5 cargo contract types handy,supra,ultra,pana,kamsar for each port.\ 
        The function :
        * returns a 1d array of 4 elements
        * all array elements are integers from 0 to 4
        * the sum of the array must be 4
        """
        day = step
        # creating 4 contracts for the specific day
        contr_count = 4
        np.random.seed(seed)
        # x array of 5 elements that sum to one
        x = np.random.multinomial(contr_count, [1/5]*5, None)
        # y array of 45 zeros
        z = np.zeros(shape=45, dtype=np.int32)
        # contracts of size 50 concatenation of x,y
        contracts = np.concatenate((x,z))
        np.random.shuffle(contracts)
        # getting locations with non zero contracts
        locations = np.nonzero(contracts)[0]
        # getting the start ports of the contract 
        ports = np.floor_divide(locations,5)
        # ports start at port 1
        ports = ports + 1
        # getting the contract count for a specific location
        contract_count_per_loc = contracts[locations]
        # array of fives to help with modulo
        fives = np.ones((len(locations)))*5
        # find ship type at port with modulo
        x = np.mod(locations,fives)
        # creating the ship_conditions for each shiptype
        ship_conditions = [ x == 0 , x == 1, x == 2, x == 3, x == 4 ]
        ship_choices = ['handymax','supramax','ultramax','panamax','kamsarmax']
        # getting the shiptype and converting it to str
        ship_types = np.select(ship_conditions,ship_choices)
        ship_types = ship_types.astype(str)
        # populating the df
        df['ship_type'] = np.repeat(ship_types,contract_count_per_loc)
        df['start_port'] = np.repeat(ports,contract_count_per_loc)
        df['end_port'] = np.random.randint(low=1, high=11, size=(4,))
        # Check for contracts with same start and end ports
        same_ports = df['start_port'] == df['end_port']
        # repeat until no start and end ports of the same contract are the same 
        while sum(same_ports) != 0 :
          df['end_port'] = np.where(same_ports, np.random.randint(low=1, high=11, size=same_ports.shape), df['end_port'])
          same_ports = df['start_port'] == df['end_port']

        # get distance between start and end ports
        start_ports_index = df['start_port'] - 1
        end_ports_index = df['end_port']
        dist_df = dm.iloc[start_ports_index,end_ports_index]
        # the distance 
        df['distance'] = pd.Series(np.diag(dist_df)).reindex()


        # Create cargo size based on ship_type
        type_conditions = [ df['ship_type'] == 'handymax', 
                            df['ship_type'] == 'supramax',
                            df['ship_type'] == 'ultramax',
                            df['ship_type'] == 'panamax',
                            df['ship_type'] == 'kamsarmax']

        cargo_size_choices = [np.random.randint(30_000,40_000,type_conditions[0].shape),
                              np.random.randint(40_000,50_000,type_conditions[1].shape),
                              np.random.randint(50_000,60_000,type_conditions[2].shape),
                              np.random.randint(60_000,70_000,type_conditions[3].shape),
                              np.random.randint(70_000,80_000,type_conditions[4].shape)]

        df['cargo_size'] = np.select(type_conditions,cargo_size_choices)
        
        # add contract value 
        df['value'] = df['cargo_size'] * 2

        # add start day
        df['start_day'] = day
        print(df) 


        # calc end day with duration


In [236]:
create_contrs()

   start_port  end_port ship_type  start_day end_day assigned_ship   value  \
0           2         9   panamax          1     NaN           NaN  137338   
1           5         3  ultramax          1     NaN           NaN  114186   
2           5        10  ultramax          1     NaN           NaN  100044   
3           6         4  handymax          1     NaN           NaN   73902   

   cargo_size duration  distance  
0       68669      NaN      2764  
1       57093      NaN       370  
2       50022      NaN       425  
3       36951      NaN       356  


In [222]:
s = np.array(['handymax','handymax','ultramax','panamax','kamsarmax'])
cond = [s == 'handymax',
        s =='supramax',
        s =='ultramax',
        s == 'panamax',
        s == 'kamsarmax']
y = [np.random.randint(30_000,40_000,),
    np.random.randint(40_000,50_000),
    np.random.randint(50_000,60_000),
    np.random.randint(60_000,70_000),
    np.random.randint(70_000,80_000)]


print(sum(cond[0]))
print(y)
l = np.select(cond,y)
print(l)

2
[30694, 49568, 51337, 63297, 79189]
[30694 30694 51337 63297 79189]


In [224]:
s = np.array(['handymax','handymax','ultramax','panamax','kamsarmax'])
handy = s == 'handymax'
supra = s == 'supramax'
ultra = s == 'ultramax'
pana = s == 'panamax'
kamsar = s == 'kamsarmax'

l = []
for i in range(5):
    

AttributeError: 'list' object has no attribute 'sum'

In [ ]:
cargo_size = np.array()

In [202]:
create_contrs()

   start_port  end_port ship_type start_day end_day assigned_ship value  \
0           1         9  ultramax       NaN     NaN           NaN   NaN   
1           5         2  supramax       NaN     NaN           NaN   NaN   
2           5         9  supramax       NaN     NaN           NaN   NaN   
3           7         2  supramax       NaN     NaN           NaN   NaN   

   cargo_size duration  distance  
0       68680      NaN       696  
1       39137      NaN      1543  
2       75086      NaN      1480  
3       36858      NaN      2503  


In [172]:
# get_distance(dm, start_ports, end_ports)
dm1 = distance_matrix
start_ports = np.array([6,6,8,9])
end_ports = np.array([9,9,5,1])
start_id = start_ports - 1
end_id = end_ports
x = dm1.iloc[start_id,end_id]
pd.Series(np.diag(x))


0    412
1    412
2     83
3    696
dtype: int64

In [170]:
start_id = start_ports - 1
end_id = end_ports

In [135]:
dm1.iloc[start_ports_index,end_ports_index]

,Xiamen,Guangzhou,Xiamen,Xiamen
0,585,928,585,585
4,370,0,370,370
8,1147,1480,1147,1147
9,124,425,124,124


In [116]:
create_contrs()

   start_port  end_port  ship_type start_day end_day assigned_ship value size  \
1           1         4    panamax       NaN     NaN           NaN   NaN  NaN   
2           5         6   supramax       NaN     NaN           NaN   NaN  NaN   
3           9         4   supramax       NaN     NaN           NaN   NaN  NaN   
4          10         4  kamsarmax       NaN     NaN           NaN   NaN  NaN   

  duration distance  
1      NaN      NaN  
2      NaN      NaN  
3      NaN      NaN  
4      NaN      NaN  

1    False
2    False
3    False
4    False
dtype: bool

0

   start_port  end_port  ship_type start_day end_day assigned_ship value size  \
1           1         4    panamax       NaN     NaN           NaN   NaN  NaN   
2           5         6   supramax       NaN     NaN           NaN   NaN  NaN   
3           9         4   supramax       NaN     NaN           NaN   NaN  NaN   
4          10         4  kamsarmax       NaN     NaN           NaN   NaN  NaN   

  duration distan

In [99]:
a = np.array([1,2,3])
b = np.array([1,0,3])
d = a == b
print(d)
k = np.equal(a,b)
print(k)

[ True False  True]
[ True False  True]


In [100]:
while sum(np.equal(a,b)) != 0 :
    print(sum(np.not_equal(a,b)))
    print(a)
    print(b)
    b = np.where(d, np.random.randint(low=1, high=11, size=d.shape), b)
    print(b)
    


1
[1 2 3]
[1 0 3]
[4 0 2]


In [78]:
np.arange(1,12)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [101]:
c_df = contracts_df.copy()

In [64]:
np.random.randint(low=1, high=11, size=(4,))

array([9, 3, 4, 9])

In [120]:
x = np.arange(start=0,stop=1095,step=3)
print(x)
print(len(x)) # 365
y = np.delete(x,[0])
y = np.insert()
print(y)
print(len(y)) # 364
print(x)
print(len(x)) # 365

[   0    3    6    9   12   15   18   21   24   27   30   33   36   39
   42   45   48   51   54   57   60   63   66   69   72   75   78   81
   84   87   90   93   96   99  102  105  108  111  114  117  120  123
  126  129  132  135  138  141  144  147  150  153  156  159  162  165
  168  171  174  177  180  183  186  189  192  195  198  201  204  207
  210  213  216  219  222  225  228  231  234  237  240  243  246  249
  252  255  258  261  264  267  270  273  276  279  282  285  288  291
  294  297  300  303  306  309  312  315  318  321  324  327  330  333
  336  339  342  345  348  351  354  357  360  363  366  369  372  375
  378  381  384  387  390  393  396  399  402  405  408  411  414  417
  420  423  426  429  432  435  438  441  444  447  450  453  456  459
  462  465  468  471  474  477  480  483  486  489  492  495  498  501
  504  507  510  513  516  519  522  525  528  531  534  537  540  543
  546  549  552  555  558  561  564  567  570  573  576  579  582  585
  588 

In [112]:
step = 1
lower = 
upper = 4

In [113]:
c_df[step-1:4*step][['start_port','ship_type']]

,start_port,ship_type
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN


In [104]:
populate_contracts_df()

[['port_3' 'supramax']
 ['port_7' 'panamax']
 ['port_10' 'supramax']
 ['port_10' 'panamax']]


In [95]:
sy = np.array([0,1,1,0,0,1,0,1,0,0])
print(f"contracts vector is {sy}")
locs = np.nonzero(sy)[0]
print(f"Non zero contracts locations are {locs}")
prts = np.floor_divide(locs,5)
print(f"Contract locations are ports {prts+1}")
contract_count_per_loc = sy[locs]
print(f"Number of contracts for each contract location are {contract_count_per_loc}")
fives = np.ones((len(locs)))*5
# find ship type at port with mode
x = np.mod(locs,fives)
#print(f"Dissered ship type code is {x}")
conditions = [ x == 0 , x == 1, x == 2, x == 3, x == 4 ]
choices = ['handymax','supramax','ultramax','panamax','kamsarmax']
ship_types = np.select(conditions,choices)
ship_types = ship_types.astype(str)
print(f"Dissered ship types are {ship_types}")


# desired output 
# port_1, supra
# port_1, supra
# port_2, pana
# port_2, pana


contracts vector is [0 1 1 0 0 1 0 1 0 0]
Non zero contracts locations are [1 2 5 7]
Contract locations are ports [1 1 2 2]
Number of contracts for each contract location are [1 1 1 1]
Dissered ship types are ['supramax' 'ultramax' 'handymax' 'ultramax']


[['port_1' 'supramax']
 ['port_1' 'ultramax']
 ['port_2' 'handymax']
 ['port_2' 'ultramax']]


In [105]:
def get_ship_type(contrs):
    pass


In [38]:
import gym 
import pandas as pd
import numpy as np
from gym import spaces
from utils.utils import cii_expected

class CarbonEnv(gym.Env):
    """
    Description :

        A custom openai gym environment for the carbon emission problem.

    """


    def __init__(self,data_dict):
        super().__init__()

        NUM_CONTRACTS_PER_STEP = 4
        NUM_SHIPS = 10
        SET_OF_SPEEDS = [10,12,14]
        MIN_REVENUE = 1_000_000
       # TERMINATION_FROM_CII = # AT LEAST 5 SHIPS DROP THEIR RATING

        self.fleet = pd.read_csv(data_dict['fleet_path'])
        self.ports = pd.read_csv(data_dict['ports_path'])
        self.dm = pd.read_csv(data_dict['dm_path'])

        # calculate fleet's required cii
        self.fleet['cii_required'] = self.fleet['dwt'].map(cii_expected)

        # set fleet at random ports
        self.fleet['current_port'] = np.random.randint(1,11,self.fleet.shape[0])

        self.state = None
        self.previous_action = None



        observation_space = spaces.Dict({
            "daily_contracts" : spaces.Discrete(NUM_CONTRACTS_PER_STEP,), # 4 new contracts each day
            "available_ships" : spaces.Box(low=0,high=1,shape=(NUM_SHIPS,),dtype=np.int32) # [0,1,1,0,0,1,0,1,1,0]
                                                                                            # ships: 1,2,5,7,8 are available
        })

        action_space = spaces.Dict({
            "choose_contract" : spaces.Box(low=0,high=1,shape=(4,),dtype=np.int32), # (0,0,1,1) deny contracts 0,1 accept contracts 2,3
            "choose_ship" : spaces.Box(low=0, high=10, shape=(4,),dtype=np.int32), # (0,0,5,7) assign ships 5,7 to contracts 2,3
            "choose_speed" : spaces.Box(low=0, high=3, shape=(4,),dtype=np.int32) # (0,0,1,2) 10 knots for ship 5, 12knots for ship 7  
            })


        self.reset()

    def create_contracts(self, seed=None):
        """
        A method that creates 4 random cargo contracts at each env step. \ 
        There are 5 cargo contract types handy,supra,ultra,pana,kamsar for each port.\ 
        The function :
        * returns a 1d array of 50 elements
        * all array elements are integers from 0 to 4
        * the sum of the array must be 4
        """
        contract_count = 4
        np.random.seed(seed)
        x = np.random.multinomial(contract_count, [1/5]*5, None)
        z = np.zeros(shape=45, dtype=np.int32)
        c = np.concatenate((x,z))
        np.random.shuffle(c)
        return c




    def step(self, action):
        # Execute one time step in the environment based on the action chosen by the model
        # action will have shape 3x4
        print(len(action))
        print(action)
        pass

    def reset(self):
        # Set the state of the environment to an initial state
        # return self._RESET()
        pass

    def _STEP(self,action_dict):
        pass
        
    def _RESET(self):
        #return self.state
        pass





    pass




In [39]:
envo = CarbonEnv(data_dict)

In [ ]:
action = {
    np.random.randint()
}

In [ ]:
np.array([[1,2],[3,4],[5,6

In [40]:
envo.step()

TypeError: step() missing 1 required positional argument: 'action'

In [41]:
envo.reset()

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0])

In [19]:
x = np.array([[0,1],
            [2,3],
            [4,5]])

In [ ]:
# training loop schema

agent = GraphAttentionModel


years = 3000 
for i in range(years): 
    obs = env.reset()
    done = False 
    score = 0
    day = 1
    while not done: # while year is not done
        act = agent.choose_action(obs)
        new_state, reward, done, info = env.step(act)
        agent.remember(obs,act, reward, new_state, int(done))
        agent.learn()
        score += reward
        day += 1
        obs = new_state

